# Week6 - Classifier Evaluation Lab

* Copy&paste your model for homework5 model
* Add grid search and train
* Compare performance
* Which one is better? Explain?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/classification/loan_status_data/loan_status.csv')

In [4]:
numerical_vars = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']
categorical_vars = ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
                        'Self_Employed', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']

In [5]:
from sklearn.model_selection import train_test_split
X = df.drop('Loan_Status', axis=1)  
y = df['Loan_Status'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=120)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']
categorical_columns = ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
                        'Self_Employed', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']


numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', StandardScaler()) 
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  
])

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_pipeline, numerical_columns),
        ('categorical', categorical_pipeline, categorical_columns)
    ])


X_train_transformed = preprocessor.fit_transform(X_train)


print("Shape of Numerical Pipeline Output:", X_train_transformed.shape)

Shape of Numerical Pipeline Output: (304, 334)


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=120))
])

param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'classifier__penalty': ['l1', 'l2']  # Penalty term
}

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Grid search with pipeline
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Select best model
best_model = grid_search.best_estimator_

# Evaluate best model
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Best model accuracy on test set:", test_accuracy)

Best model accuracy on test set: 0.8181818181818182


C:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\lib\site-packages\sklearn\base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Anaconda\lib\site-packages\sklearn\pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "C:\Anaconda\lib\site-packages\sklearn\base.py", line 1351, 

Before grid search, the model showed 87% accuracy, but it might have been too focused on the training data. After grid search, even though the accuracy dropped slightly to 82%, it's a more reliable measure as it's less likely to overfit and better generalizes to new data.